In [310]:
import os
import requests

In [312]:
#Get pdf path
pdf_path = "../data/human_nutrition.pdf"

if not os.path.exists(pdf_path):
    print(f"File not found at {pdf_path}, downloading...")

    #The url of the pdf we want to download
    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download"

    #Get the only the filename from the path
    filename = pdf_path.split("/")[-1]

    #send a GET request to the url
    response = requests.get(url)

    if response.status_code == 200:
        #open the file in binary write mode and save the content to it
        with open(filename, "wb") as file:
            file.write(response.content)
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

else:
    print(f"File {pdf_path} does exist")

File ../data/human_nutrition.pdf does exist


PDF Text EXTRACTION

In [314]:
import fitz  #PyMuPDF - for scraping pdfs - only for digital formats
from tqdm.auto import tqdm

def text_formatter(text:str) -> str:
    """ Performs minor formatting on text"""
    #replace newlines with spaces and strip whitespace
    cleaned_text = text.replace("\n", " ").strip()

    # other potential textformatting
    return cleaned_text

def open_and_read_pdf(pdf_path:str) -> list:
    """Opens a pdf, reads its text page by page, and collects statistics.
    
    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count 
        and the extracted text for each page.
    """
    #open the pdf file
    doc = fitz.open(pdf_path) #open a document with PyMuPDF

    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text() # get plain text encoded as UTF-8
        text = text_formatter(text) #format the text
        pages_and_texts.append({"page_number": page_number - 41, #adjust page number since our PDF starts at page 42
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count":len(text)/4, # 1 token = ~4 chars
                                "text": text})
    
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path)


0it [00:00, ?it/s]

In [317]:
import random 
random.sample(pages_and_texts, k=3)

[{'page_number': 406,
  'page_char_count': 1662,
  'page_word_count': 299,
  'page_sentence_count_raw': 11,
  'page_token_count': 415.5,
  'text': 'amino acids at adequate levels. Growing children and the elderly  need to ensure they get enough protein in their diet to help build  and maintain muscle strength. Even if you’re a hardcore athlete, get  your proteins from nutrient-dense foods as you need more than  just protein to power up for an event. Nuts are one nutrient-dense  food with a whole lot of protein. One ounce of pistachios, which  is about fifty nuts, has the same amount of protein as an egg and  contains a lot of vitamins, minerals, healthy polyunsaturated fats,  and antioxidants. Moreover, the FDA says that eating one ounce of  nuts per day can lower your risk for heart disease. Can you be a  hardcore athlete and a vegetarian?  The analysis of vegetarian diets by the Dietary Guidelines  Advisory Committee (DGAC) did not find professional athletes were  inadequate in any n

Chunking Strategy 1: FIXED SIZE Chunking

In [319]:
def chunk_text(text:str, chunk_size:int = 500) -> list:
    """
    Splits text into chunk of approx 'chunk_size' characters.
    """

    chunks = []
    current_chunk = ''
    words = text.split() #list of words

    for word in words:
        #Check if addding the word exceeds the chunk size
        if(len(current_chunk) + len(word) + 1 <= chunk_size):
            current_chunk += (word + " ")
        else:
            #store the finished chunk and start a new one
            chunks.append(current_chunk.strip())
            current_chunk = word + " "

    #add the last chunk
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def chunk_pdf_pages(pages_and_texts:list, chunk_size:int = 500) -> list:
    """
    Takes pdf pages with text and splits them into chunks.
    Returns a list of dicts with page_number, chunk_index, and chunk_text.
    """

    all_chunks = []
    for page in pages_and_texts:
        page_number = page["page_number"]
        page_text = page["text"]

        chunks = chunk_text(page_text, chunk_size)
        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i,
                "chunk_char_count": len(chunk),
                "chunk_word_count": len(chunk.split()),
                "chunk_token_count": len(chunk)/4,  #token estimated
                "chunk_text": chunk
            })
        
    return all_chunks

#Exemple usage
all_chunks = chunk_pdf_pages(pages_and_texts = pages_and_texts, chunk_size = 500)
print(f"Total chunks: {len(all_chunks)}")
print(f"First chunk ( page {all_chunks[0]['page_number']}): {all_chunks[0]['chunk_text'][:100]}")




Total chunks: 3321
First chunk ( page -41): Human Nutrition: 2020 Edition


In [325]:
import random, textwrap

#------------Sampling & Pretty Printing------------

def _scattered_indices(n: int, k: int, jitter_frac: float = 0.08) -> list:
    """Evenly spaced anchors + random jitter -> indices scattered across [0, n-1]"""

    if k <= 0:
        return []
    if k == 1:
        return [random.randrange(n)]
    anchors = [int(round(i * (n - 1) / (k - 1))) for i in range(k)]
    out,seen = [],set()
    radius = max(1, int(jitter_frac))
    for a in anchors:
        lo, hi = max(0, a - radius), min(n - 1, a + radius)
        j = random.randint(lo, hi)
        if j not in seen:
            out.append(j); seen.add(j)
    while len(out) < k:
        r = random.randrange(n)
        if r not in seen:
            out.append(r); seen.add(r)
    return out

def _draw_boxed_chunk(c: dict, wrap_at: int = 96) -> str:
    header = (
        f" Chunk p{c['page_number']} - idx {c['chunk_index']}  |  "
        f"chars {c['chunk_char_count']} - words {c['chunk_word_count']} -  ~tokens {c['chunk_token_count']} "
    )

    wrapped_lines = textwrap.wrap(
        c["chunk_text"], width=wrap_at, break_long_words = False,replace_whitespace = False
        )
    context_width = max([0, *map(len, wrapped_lines)])
    box_width = max(len(header), context_width + 2)

    top = "||" + "=" * (box_width) + "||"
    hline = "||" + header.ljust(box_width) + "||"
    sep = "||" + "-" * (box_width) + "||"
    body = "\n".join("|| " + line.ljust(box_width - 2) + " ||" for line in wrapped_lines) or \
        ("|| " + "".ljust(box_width - 2) + " ||")
    bottom = "||" + "=" * (box_width) + "||"
    return "\n".join([top, hline, sep, body, bottom])


def show_random_chunks(pages_and_texts:list, chunk_size:int = 500, k: int = 5, seed: int | None = 42):
    if seed is not None:
        random.seed(seed)
    all_chunks = chunk_pdf_pages(pages_and_texts, chunk_size)
    if not all_chunks:
        print("No chunks to display.");
        return
    idxs = _scattered_indices(len(all_chunks), k)
    print(f"Showing {len(idxs)} scattered random chunks out of {len(all_chunks)} total: \n")
    for i, idx in enumerate(idxs,1):
        print(f"#{i}")
        print(_draw_boxed_chunk(all_chunks[idx]))
        print()


#---------Run--------

assert 'pages_and_texts' in globals(), "Run: pages_and_texts = open_and_read_pdf(pdf_path = pdf_path) first"
show_random_chunks(pages_and_texts, chunk_size = 500, k = 5, seed = 42)


Showing 5 scattered random chunks out of 3321 total: 

#1
||===========================================================||
|| Chunk p-41 - idx 0  |  chars 29 - words 4 -  ~tokens 7.25 ||
||-----------------------------------------------------------||
|| Human Nutrition: 2020 Edition                             ||
||===========================================================||

#2
||=================================================================================================||
|| Chunk p279 - idx 2  |  chars 499 - words 71 -  ~tokens 124.75                                   ||
||-------------------------------------------------------------------------------------------------||
|| consuming products with artificial sweeteners decreases weight. In fact, some studies suggest   ||
|| the intense sweetness of these products increases appetite for sweet foods and may lead to      ||
|| increased weight gain. Also, there is very limited evidence that suggests artificial sweeteners ||
|| low

Chunking Strategy 2: SEMANTIC Chunking

In [333]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np
import nltk
nltk.download('punkt_tab')
#Load once globally
semantic_model = SentenceTransformer("all-MiniLM-L6-v2")
 
def semantic_chunk_text(text: str, similarity_threshold: float = 0.8, max_tokens: int = 500)-> list:
    """
    Splits text into semantic chunks based on sentence similarity and max token lenght
    """
    #break the page into sentences
    sentences = nltk.sent_tokenize(text) #list
    if not sentences:
        return []
    
    embeddings = semantic_model.encode(sentences)
    
    chunks = []
    current_chunk = [sentences[0]]
    current_embeddings  = embeddings[0]

    for i in range(1, len(sentences)):
        #[0][0] takes the value of the cosine similarity
        sim = cosine_similarity([current_embeddings], [embeddings[i]])[0][0]
        chunk_token_count = len(" ".join(current_chunk)) // 4

        if sim >= similarity_threshold and chunk_token_count < max_tokens:
            current_chunk.append(sentences[i])
            #mean of the embeddings chunk
            #I keep only the mean of the embeedings of the chunk(not all the embeddings)
            #each chunk will have only one embedding(the mean of the similar embeddings)
            current_embeddings = np.mean([current_embeddings, embeddings[i]], axis = 0)
        else:
            chunks.append(" ".join(current_chunk)) #string with the current chunk
            current_chunk = [sentences[i]]
            current_embeddings = embeddings[i]

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks
                                
    
from tqdm.auto import tqdm

def semantic_chunk_pdf_pages(pages_and_texts:list, similarity_threshold: float = 0.8, max_tokens: int = 500) -> list:
    """
    Takes pdf pages with text and splits them into semantic chunks.
    Returns a list of dicts with page_number, chunk_index, and chunk_text.
    """

    all_chunks = []
    for page in tqdm(pages_and_texts, desc = "Semantic Chunking"):
        page_number = page["page_number"]
        page_text = page["text"]

        chunks = semantic_chunk_text(page_text, similarity_threshold, max_tokens)

        for i,chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i,
                "chunk_char_count": len(chunk),
                "chunk_word_count": len(chunk.split()),
                "chunk_token_count": len(chunk)/4,
                "chunk_text": chunk
            })

    return all_chunks


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/tybyboiciuc/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
2025-10-04 16:57:16,262 - INFO - Use pytorch device_name: mps
2025-10-04 16:57:16,263 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [334]:
import nltk

semantic_chunk_pages = semantic_chunk_pdf_pages(pages_and_texts, similarity_threshold = 0.75, max_tokens = 500)

print(f"Total semantic chunks: {len(semantic_chunk_pages)}")
print(f"First semantic chunk page {semantic_chunk_pages[0]['page_number']}")
print(semantic_chunk_pages[0]['chunk_text'][:200])

Semantic Chunking:   0%|          | 0/1208 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Total semantic chunks: 12016
First semantic chunk page -41
Human Nutrition: 2020 Edition


In [ ]:
import torch
import torch.nn.functional as F
device = "mps" if torch.backends.mps.is_available() else "cpu"
semantic_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)

def semantic_chunk_text_GPU(text: str, similarity_threshold: float = 0.8, max_tokens: int = 500)-> list:
    """
    Splits text into semantic chunks based on sentence similarity and max token length
    """
    #break the page into sentences
    sentences = nltk.sent_tokenize(text) #list
    if not sentences:
        return []
    
    embeddings = semantic_model.encode(sentences, convert_to_tensor=True, device=device)
    
    chunks = []
    current_chunk = [sentences[0]]
    current_embeddings  = embeddings[0]

    for i in range(1, len(sentences)):
        #[0][0] takes the value of the cosine similarity
        sim = F.cosine_similarity(current_embeddings, embeddings[i], dim=0).item()
        chunk_token_count = len(" ".join(current_chunk)) // 4
        

        if sim >= similarity_threshold and chunk_token_count < max_tokens:
            current_chunk.append(sentences[i])
            #mean of the embeddings chunk
            #I keep only the mean of the embeedings of the chunk(not all the embeddings)
            #each chunk will have only one embedding(the mean of the similar embeddings)
            current_embeddings = torch.mean(torch.stack([current_embeddings, embeddings[i]]), dim=0)
        else:
            chunks.append(" ".join(current_chunk)) #string with the current chunk
            current_chunk = [sentences[i]]
            current_embeddings = embeddings[i]

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks
                                
    
from tqdm.auto import tqdm

def semantic_chunk_pdf_pages_GPU(pages_and_texts:list, similarity_threshold: float = 0.8, max_tokens: int = 500) -> list:
    """
    Takes pdf pages with text and splits them into semantic chunks.
    Returns a list of dicts with page_number, chunk_index, and chunk_text.
    """

    all_chunks = []
    for page in tqdm(pages_and_texts, desc = "Semantic Chunking"):
        page_number = page["page_number"]
        page_text = page["text"]

        chunks = semantic_chunk_text_GPU(page_text, similarity_threshold, max_tokens)

        for i,chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i,
                "chunk_char_count": len(chunk),
                "chunk_word_count": len(chunk.split()),
                "chunk_token_count": len(chunk)/4,
                "chunk_text": chunk
            })

    return all_chunks

2025-10-04 16:45:50,096 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


TypeError: list indices must be integers or slices, not dict

In [ ]:
semnatic_chunk_pages_GPU = semantic_chunk_pdf_pages_GPU(pages_and_texts, similarity_threshold = 0.75, max_tokens = 500)

print(f"Total semantic chunks: {len(semnatic_chunk_pages_GPU)}")
print(f"First semantic chunk page {semnatic_chunk_pages_GPU[0]['page_number']}")
print(semnatic_chunk_pages_GPU[0]['chunk_text'][:200])

Semantic Chunking:   0%|          | 0/1208 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

Method 3: RECURSIVE STRUCTURED Chunking -> DOCLING

In [340]:
from docling.document_converter import DocumentConverter


def extract_pdf_with_docling(pdf_path:str) -> str:
    """
    Extracts the text from a pdf file using the docling library.
    """
    try:
        if not os.path.exists(pdf_path):
            raise FileNotFoundError(f"File not found: {pdf_path}")
        
        converter = DocumentConverter()
        result = converter.convert(pdf_path)
        markdown_text = result.document.export_to_markdown()
        return markdown_text
    except Exception as e:
        print(f"Error extracting PDF with Docling: {e}")
        return ""

pdf_path = "../data/human_nutrition.pdf"
markdown_text = extract_pdf_with_docling(pdf_path)

2025-10-04 17:06:20,595 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-04 17:06:20,652 - INFO - Going to convert document batch...
2025-10-04 17:06:20,653 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-10-04 17:06:20,654 - INFO - Accelerator device: 'mps'
2025-10-04 17:06:25,282 - INFO - Accelerator device: 'mps'
2025-10-04 17:06:26,682 - INFO - Accelerator device: 'mps'
2025-10-04 17:06:27,321 - INFO - Processing document human_nutrition.pdf
2025-10-04 17:18:10,962 - INFO - Finished converting document human_nutrition.pdf in 503.04 sec.


In [ ]:
import nltk
from tqdm.auto import tqdm



def recursive_chunk_text(text:str, max_chunk_size:int = 1000) -> list:
    """
    Recursively splits a block of text into chunks that fit within size constraints.
    Tries splitting by sections, then newlines, then sentences.
    """
    def split_chunk(chunk: str) -> list:
        if len(chunk) <= max_chunk_size:
            return [chunk]
    
        # Try splitting by sections
        sections = chunk.split("##")
        if len(sections) > 1:
            result = []
            for section in sections:
                if section.strip():
                    # print(f"Split chunk: {section.strip()}")
                    result.extend(split_chunk(section.strip()))
            return result
        
        # Try splitting by single newline
        sections = chunk.split("\n")
        if len(sections) > 1:
            result = []
            for section in sections:
                if section.strip():
                    # print(f"Split chunk by newline: {section.strip()}")
                    result.extend(split_chunk(section.strip()))
            return result
        
        # Fallback: split by sentences
        sentences = nltk.sent_tokenize(chunk) #list of sentences
        chunks, current_chunk, current_size = [], [], 0
        #chunk will be the final list of chunks
        #current_chunk will be the temporary chunk that we are building and adding to the final list
        
        for sentence in sentences:
            if current_size + len(sentence) > max_chunk_size:
                if current_chunk:
                    chunks.append(" ".join(current_chunk)) #" ".join(current_chunk) will join the 
                                                    # entire list into a string(one element)
                current_chunk = [sentence]
                current_size = len(sentence)
            else:
                current_chunk.append(sentence)
                current_size += len(sentence)
        
        if current_chunk:
            chunks.append(" ".join(current_chunk))
            
        return chunks

    return split_chunk(text)


In [337]:
import nltk
from tqdm.auto import tqdm



def recursive_chunk_text(text:str, max_chunk_size:int = 1000) -> list:
    """
    Recursively splits a block of text into chunks that fit within size constraints.
    Tries splitting by sections, then newlines, then sentences.
    """
    def split_chunk(chunk: str) -> list:
        if len(chunk) <= max_chunk_size:
            return [chunk]
    
        # Try splitting by sections
        sections = chunk.split("##")
        if len(sections) > 1:
            result = []
            for section in sections:
                if section.strip():
                    # print(f"Split chunk: {section.strip()}")
                    result.extend(split_chunk(section.strip()))
            return result
        
        # Try splitting by single newline
        sections = chunk.split("\n")
        if len(sections) > 1:
            result = []
            for section in sections:
                if section.strip():
                    # print(f"Split chunk by newline: {section.strip()}")
                    result.extend(split_chunk(section.strip()))
            return result
        
        # Fallback: split by sentences
        sentences = nltk.sent_tokenize(chunk) #list of sentences
        chunks, current_chunk, current_size = [], [], 0
        #chunk will be the final list of chunks
        #current_chunk will be the temporary chunk that we are building and adding to the final list
        
        for sentence in sentences:
            if current_size + len(sentence) > max_chunk_size:
                if current_chunk:
                    chunks.append(" ".join(current_chunk)) #" ".join(current_chunk) will join the 
                                                    # entire list into a string(one element)
                current_chunk = [sentence]
                current_size = len(sentence)
            else:
                current_chunk.append(sentence)
                current_size += len(sentence)
        
        if current_chunk:
            chunks.append(" ".join(current_chunk))
            
        return chunks

    return split_chunk(text)


In [338]:
recursive_chunked_text = recursive_chunk_text(markdown_text)
print(f"length of the recursive_chunked_text: {len(recursive_chunked_text)}")
print(f"First chunk length: {len(recursive_chunked_text[0])}")
print(f"Last chunk length: {len(recursive_chunked_text[-1])}")


length of the recursive_chunked_text: 6459
First chunk length: 45
Last chunk length: 132


In [348]:
def combine_small_chunks_optimized(chunks: list, min_chunk_size: int = 200) -> list:
    """Combine small chunks(consecutive titles and subtitles) into larger chunks"""
    if not chunks:
        return []
    
    result = []
    current_combined = ""
    
    for chunk in chunks:
        if len(chunk) < min_chunk_size:
            # Combine with the current chunk
            if current_combined:    
                current_combined += " " + chunk
            else:
                current_combined = chunk
        else:
            # The chunk is large, save the combined chunk and add the large chunk
            if current_combined:
                result.append(current_combined)
                current_combined = ""
            result.append(chunk)
    
    # Save the last combined chunk
    if current_combined:
        result.append(current_combined)
    
    return result

combined_chunks = combine_small_chunks_optimized(recursive_chunked_text)
print(f"Number of chunks after combining: {len(combined_chunks)}")
print(f"First chunk: {combined_chunks[0]}")
print(f"Last chunk: {combined_chunks[-1]}")




Number of chunks after combining: 3291
First chunk: Human Nutrition: 2020 Edition

<!-- image -->
Last chunk: 13. Pasteurization reused 'Simple Pasteurization' by Emma / Attribution-ShareAlike 14. Polychlorated Biphenyls reused in 'Polychlorated Biphenyl structure' by D.328 / Public Domain;  'PCBs in power transformer' by Sturmovik / Attribution-ShareAlike 15. Protein reused 'The Macronutrients: Carbohydrates, Lipids, Protein, and Water' by Medicine LibreTexts / AttributionShareAlike 16. Potential Energy reused 'Silhouette Wheel Cyclist Bike' by mohamed\_hassan / Pixabay License 17. Salivary Amylase reused 'Salivary glands' by Medicine LibreTexts / Attribution-ShareAlike 18. Soluble Fiber/Unsoluble Fiber reused 'List of High Fiber Foods and Their Health Benefits' by Urban Wired / Attribution Sharealike 19. Solutes reused 'Syringe Doctor Needle' by janjf93 / Pixabay License; 'Flask Beaker School Laboratory' by UnboxScience / Pixabay License 20. Sucrose reused 'Figure 03 02 05' by OpenSt